![logo](https://neutouch.eu/templates/yootheme/cache/g37-01577415.png)
# NeuTouch summer school on Touch and Robotics
Tutorial: Slip Detection with Neural Networks

In this tutorial we will look into incipient slip detection and slip classification with deep neural networks. We will experiment with and evaluate various neural network architectures to detect and classify slippage.
To this end, we will work on a pre-recorded data set from our 16x16 tactile sensor array, comprising tactile time series data for three different situations:
- stable grasp condition
- translational slip
- rotational slip


In [ ]:
# Let's download the data set first:
!wget "https://uni-bielefeld.sciebo.de/s/7vi5lX9WO1VmvIZ/download" -O "data_stable_slip_rotate.pkl"

In [ ]:
# Actually load the data
import pickle

data = pickle.load(open("data_stable_slip_rotate.pkl", "rb"), encoding='latin1')

## Task: Visually inspect the data

Use [matplotlib's animation](https://matplotlib.org/stable/api/animation_api.html) to create videos for all three classes (stable, translation, rotation)
and [embed them into the notebook](http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# The data is organized into three classes:
fig = plt.figure()  # create a new figure
for idx, key in enumerate(data.keys()):
    ax = fig.add_subplot(1, 3, idx+1, xticks=[], yticks=[],  # add subplot w/o any axis ticks
                         xlabel="{name}: {shape}".format(name=key, shape=data[key].shape))
    ax.imshow(data[key][np.random.randint(0, len(data[key]))], cmap = "Greys")  # plot a random sample

## Baseline CNN classification of raw data

As a baseline approach, we illustrate here how to use a CNN to classify the raw data.
As seen above, each class comprises 50k frames of consecutive sensor recordings, recorded at a frame rate of 1kHz. Hence, we have 50s of data for each class.
Obviously, the network cannot predict from a single sample whether there is slippage or not. So, let's chop the whole time-series into short sequences of fixed length, say 32 samples, which then can be fed into a neural network.

In [ ]:
# Import Keras + TensorFlow packages
import tensorflow as tf
from tensorflow import keras as keras

# Define a random seed to have deterministic results
np.random.seed(11)

In [ ]:
# Prepare the dataset for training: chop into sequences of given length and assign numeric class labels
def prepare_dataset(length=32, stride=8):
   # map textual labels onto numeric class labels
   label_mapping = dict(stable=0, translation=1, rotation=2)
   xs = []
   ys = []
   for key in data.keys():
      N = len(data[key])
      labels = np.full(N, label_mapping[key]) # generate array with identical numeric labels for key
      # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/TimeseriesGenerator
      x, y = keras.preprocessing.sequence.TimeseriesGenerator(data[key], labels, length=length, stride=stride, batch_size=N)[0]
      print(key, x.shape, y.shape)
      xs.append(x)
      ys.append(y)

   # combine data of all classes into a single data set
   X = np.concatenate(xs) / 4096  # normalize 12bit ADC data (0..2^12) into range (0..1)
   Y = np.concatenate(ys)
   print("combined", X.shape, Y.shape)

   # optionally perform FFT on input data X
   # https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html
   return X, Y

X, Y = prepare_dataset()


In [ ]:
def unison_shuffle_arrays(x, y):
	assert len(x) == len(y)
	p = np.random.permutation(len(x))
	return x[p], y[p]  # return new permutation of x and y

In [ ]:
def train_test_split(X, Y, ratio=0.8):
   split=int(len(X)*ratio)
   X_train = X[:split]
   Y_train = Y[:split]

   X_test = X[split:]
   Y_test = Y[split:]

   return X_train, X_test, Y_train, Y_test

In [ ]:
# Simple MLP with a single hidden layer
def simple_mlp(input_shape, num_classes=3):
  model = keras.models.Sequential(name="SimpleMLP")
  model.add(keras.layers.Flatten(input_shape=input_shape))  # Flatten input tensor into single vector
  model.add(keras.layers.Dense(64, activation='relu'))  # hidden layer with 64 neurons
  model.add(keras.layers.Dense(num_classes))  # dense classification layer
  return model

# Basic network with a single CNN layer
def simple_cnn(input_shape, num_classes=3):
  model = keras.models.Sequential(name="SimpleCNN")
  model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(keras.layers.Flatten())  # Flatten tensor into single vector
  model.add(keras.layers.Dense(num_classes))  # dense classification layer
  return model


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf logs/

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(*unison_shuffle_arrays(X, Y), ratio=0.8)

def train(model):
   model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer=keras.optimizers.RMSprop(),
                 metrics=['sparse_categorical_accuracy'])
   model.summary()
 
   # Specify log directory for TensorBoard
   log_dir = "logs/" + model.name
 
   # Initialize TensorBoard
   tb = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
 
   # Fit data to the model!
   model.fit(X_train, Y_train,
             batch_size=16, epochs=10,
             validation_data=(X_test, Y_test),
             callbacks=[tb])
 
   score = model.evaluate(X_test, Y_test, verbose=False)
   print('Test loss:', score[0])
   print('Test accuracy:', score[1])

In [ ]:
train(simple_mlp(X_train.shape[1:]))
train(simple_cnn(X_train.shape[1:]))

In [ ]:
# Inspect results with TensorBoard
%tensorboard --logdir logs

## Task: Experiment!
Try various networks:
- Add more CNN layers
- Adapt size of layers, size of kernels, num of kernels
- Experiment with different optimizers: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
- Compare raw inputs vs. FFT preprocessing. Are frequencies and/or phases important?

Improve generalization
- Introduce drop-out: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
- Use batch normalization: https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
- Augment your data: https://www.tensorflow.org/tutorials/images/data_augmentation
  Which data augmentation strategies are meaningful for this type of data?